In [ ]:
import argparse
from argparse import Namespace 
import torch
from torchmdnet import calculators
from torchmdnet.calculators import External
from torchmdnet.utils import LoadFromFile, LoadFromCheckpoint, save_argparse, number
from torchmdnet import datasets, priors, models
from torchmdnet.data import DataModule
from torchmdnet.models import output_modules
from torchmdnet.models.utils import rbf_class_mapping, act_class_mapping

from moleculekit.molecule import Molecule
from torchmd.forcefields.ff_yaml import YamlForcefield
from torchmd.forcefields.forcefield import ForceField
from torchmd.forces import Forces
from torchmd.integrator import Integrator, maxwell_boltzmann
from torchmd.parameters import Parameters
from torchmd.systems import System
from torchmd.wrapper import Wrapper

from torchmdexp.nn.train import external_forces, setup_forces, setup_system, forward, loss_fn, train_model

## Load args file

In [ ]:
def get_args():
    # fmt: off
    parser = argparse.ArgumentParser(description='Training')
    parser.add_argument('--load-model', action=LoadFromCheckpoint, help='Restart training using a model checkpoint')  # keep first
    parser.add_argument('--conf', '-c', type=open, action=LoadFromFile, help='Configuration yaml file')  # keep second
    parser.add_argument('--num-epochs', default=300, type=int, help='number of epochs')
    parser.add_argument('--batch-size', default=32, type=int, help='batch size')
    parser.add_argument('--inference-batch-size', default=None, type=int, help='Batchsize for validation and tests.')
    parser.add_argument('--lr', default=1e-4, type=float, help='learning rate')
    parser.add_argument('--lr-patience', type=int, default=10, help='Patience for lr-schedule. Patience per eval-interval of validation')
    parser.add_argument('--lr-min', type=float, default=1e-6, help='Minimum learning rate before early stop')
    parser.add_argument('--lr-factor', type=float, default=0.8, help='Minimum learning rate before early stop')
    parser.add_argument('--lr-warmup-steps', type=int, default=0, help='How many steps to warm-up over. Defaults to 0 for no warm-up')
    parser.add_argument('--early-stopping-patience', type=int, default=30, help='Stop training after this many epochs without improvement')
    parser.add_argument('--weight-decay', type=float, default=0.0, help='Weight decay strength')
    parser.add_argument('--ema-alpha-y', type=float, default=1.0, help='The amount of influence of new losses on the exponential moving average of y')
    parser.add_argument('--ema-alpha-dy', type=float, default=1.0, help='The amount of influence of new losses on the exponential moving average of dy')
    parser.add_argument('--ngpus', type=int, default=-1, help='Number of GPUs, -1 use all available. Use CUDA_VISIBLE_DEVICES=1, to decide gpus')
    parser.add_argument('--num-nodes', type=int, default=1, help='Number of nodes')
    parser.add_argument('--precision', type=int, default=32, choices=[16, 32], help='Floating point precision')
    parser.add_argument('--log-dir', '-l', default='/trainings', help='log file')
    parser.add_argument('--splits', default=None, help='Npz with splits idx_train, idx_val, idx_test')
    parser.add_argument('--train-size', type=number, default=None, help='Percentage/number of samples in training set (None to use all remaining samples)')
    parser.add_argument('--val-size', type=number, default=0.05, help='Percentage/number of samples in validation set (None to use all remaining samples)')
    parser.add_argument('--test-size', type=number, default=0.1, help='Percentage/number of samples in test set (None to use all remaining samples)')
    parser.add_argument('--test-interval', type=int, default=10, help='Test interval, one test per n epochs (default: 10)')
    parser.add_argument('--save-interval', type=int, default=10, help='Save interval, one save per n epochs (default: 10)')
    parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')
    parser.add_argument('--distributed-backend', default='ddp', help='Distributed backend: dp, ddp, ddp2')
    parser.add_argument('--num-workers', type=int, default=4, help='Number of workers for data prefetch')
    parser.add_argument('--redirect', type=bool, default=False, help='Redirect stdout and stderr to log_dir/log')
    
    # model architecture
    parser.add_argument('--model', type=str, default='graph-network', choices=models.__all__, help='Which model to train')
    parser.add_argument('--output-model', type=str, default='Scalar', choices=output_modules.__all__, help='The type of output model')
    parser.add_argument('--prior-model', type=str, default=None, choices=priors.__all__, help='Which prior model to use')

    # architectural args
    parser.add_argument('--embedding-dimension', type=int, default=256, help='Embedding dimension')
    parser.add_argument('--num-layers', type=int, default=6, help='Number of interaction layers in the model')
    parser.add_argument('--num-rbf', type=int, default=64, help='Number of radial basis functions in model')
    parser.add_argument('--num-filters', type=int, default=128, help='Number of filters in model')    
    parser.add_argument('--activation', type=str, default='silu', choices=list(act_class_mapping.keys()), help='Activation function')
    parser.add_argument('--rbf-type', type=str, default='expnorm', choices=list(rbf_class_mapping.keys()), help='Type of distance expansion')
    parser.add_argument('--trainable-rbf', type=bool, default=False, help='If distance expansion functions should be trainable')
    parser.add_argument('--neighbor-embedding', type=bool, default=False, help='If a neighbor embedding should be applied before interactions')
    
    # dataset specific
    parser.add_argument('--data_dir', default=None, help='Input directory')
    parser.add_argument('--dataset', default=None, type=str, choices=datasets.__all__, help='Name of the torch_geometric dataset')
    parser.add_argument('--dataset-root', default='~/data', type=str, help='Data storage directory (not used if dataset is "CG")')
    parser.add_argument('--dataset-arg', default=None, type=str, help='Additional dataset argument, e.g. target property for QM9 or molecule for MD17')
    parser.add_argument('--coord-files', default=None, type=str, help='Custom coordinate files glob')
    parser.add_argument('--embed-files', default=None, type=str, help='Custom embedding files glob')
    parser.add_argument('--energy-files', default=None, type=str, help='Custom energy files glob')
    parser.add_argument('--force-files', default=None, type=str, help='Custom force files glob')
    parser.add_argument('--energy-weight', default=1.0, type=float, help='Weighting factor for energies in the loss function')
    parser.add_argument('--force-weight', default=1.0, type=float, help='Weighting factor for forces in the loss function')

    # Transformer specific
    parser.add_argument('--distance-influence', type=str, default='both', choices=['keys', 'values', 'both', 'none'], help='Where distance information is included inside the attention')
    parser.add_argument('--attn-activation', default='silu', choices=list(act_class_mapping.keys()), help='Attention activation function')
    parser.add_argument('--num-heads', type=int, default=8, help='Number of attention heads')
    
    # Torchmdexp specific
    parser.add_argument('--device', default='cpu', help='Type of device, e.g. "cuda:1"')
    parser.add_argument('--forcefield', default="../data/ca_priors-dihedrals_general_2xweaker.yaml", help='Forcefield .yaml file')
    parser.add_argument('--forceterms', nargs='+', default="bonds", help='Forceterms to include, e.g. --forceterms Bonds LJ')
    parser.add_argument('--cutoff', default=None, type=float, help='LJ/Elec/Bond cutoff')
    parser.add_argument('--rfa', default=False, action='store_true', help='Enable reaction field approximation')
    parser.add_argument('--replicas', type=int, default=1, help='Number of different replicas to run')
    parser.add_argument('--step_update', type=int, default=5, help='Number of epochs to update the simulation steps')
    parser.add_argument('--switch_dist', default=None, type=float, help='Switching distance for LJ')
    parser.add_argument('--temperature',  default=300,type=float, help='Assign velocity from initial temperature in K')
    parser.add_argument('--force-precision', default='single', type=str, help='LJ/Elec/Bond cutoff')
    parser.add_argument('--verbose', default=None, help='Add verbose')
    parser.add_argument('--timestep', default=1, type=float, help='Timestep in fs')
    parser.add_argument('--langevin_gamma',  default=0.1,type=float, help='Langevin relaxation ps^-1')
    parser.add_argument('--langevin_temperature',  default=0,type=float, help='Temperature in K of the thermostat')
    parser.add_argument('--max_steps',type=int,default=2000,help='Total number of simulation steps')
    
    # other args
    parser.add_argument('--derivative', default=True, type=bool, help='If true, take the derivative of the prediction w.r.t coordinates')
    parser.add_argument('--cutoff-lower', type=float, default=0.0, help='Lower cutoff in model')
    parser.add_argument('--cutoff-upper', type=float, default=5.0, help='Upper cutoff in model')
    parser.add_argument('--atom-filter', type=int, default=-1, help='Only sum over atoms with Z > atom_filter')
    
    parser.add_argument('--max-z', type=int, default=100, help='Maximum atomic number that fits in the embedding matrix')
    parser.add_argument('--max-num-neighbors', type=int, default=32, help='Maximum number of neighbors to consider in the network')
    parser.add_argument('--standardize', type=bool, default=False, help='If true, multiply prediction by dataset std and add mean')
    parser.add_argument('--reduce-op', type=str, default='add', choices=['add', 'mean'], help='Reduce operation to apply to atomic predictions')

    args = parser.parse_args("")
    all_defaults = {}
    for key in vars(args):
        all_defaults[key] = parser.get_default(key)
    
    return all_defaults

In [ ]:
args = get_args()

In [ ]:
# process_yaml.py file
import yaml

with open('../arguments/cln_graph-network.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    args_file = yaml.load(file, Loader=yaml.FullLoader)
for key, value in args_file.items():
    args[key] = value
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
args = Struct(**args)

## Simulate Chingolin with pre-trained NN

Load the molecule

In [ ]:
mol = Molecule('../data/chignolin_cln025.pdb')
mol.filter('name CA')
mol.read('../data/cln_ca_top_dih.psf')
mol.read('../data/chignolin_ca_initial_coords.xtc')

2021-09-01 16:28:01,039 - moleculekit.molecule - INFO - Removed 83 atoms. 10 atoms remaining in the molecule.


Load pretrained NN

In [ ]:
from torchmdnet.models.model import load_model

netfile = '../data/epoch=47-val_loss=739.5398-test_loss=21.5975.ckpt'
model = load_model(netfile, device='cpu', derivative=True)
external = external_forces(model, mol, replicas = args.replicas, device = args.device)

Prepare the system

In [ ]:
# Define forces and parameters
forces = setup_forces(mol, args.forcefield, args.forceterms, external, device=args.device, 
                        cutoff=args.cutoff, rfa=args.rfa, switch_dist=args.switch_dist
                        )   
# System
system = setup_system(mol, forces, replicas=args.replicas, T=args.temperature,
                        device=args.device
                         )
            

Run the simulation

In [ ]:
steps = 2000
output_period = 10
# Forward pass            
native_coords, system = forward(system, forces, steps, output_period, args.timestep, device=args.device,
                                gamma=args.langevin_gamma, T=args.langevin_temperature
                               )


100%|█████████████████████████████████████████| 200/200 [01:13<00:00,  2.73it/s]


In [ ]:
loss, rmsd = loss_fn(native_coords, system.pos)
print('Final RMSD: ', rmsd)

Final RMSD:  3.966182815951867


## Simulate Chingolin with pre-trained NN trained on lambda

In [ ]:
mol = Molecule('../data/chignolin_cln025.pdb')
mol.filter('name CA')
mol.read('../data/cln_ca_top_dih.psf')
mol.read('../data/chignolin_ca_initial_coords.xtc')

2021-09-01 16:29:14,900 - moleculekit.molecule - INFO - Removed 83 atoms. 10 atoms remaining in the molecule.


In [ ]:
netfile = '/workspace2/fast_folders_cgnet/multiprotein/single_prot_fix/lambda/train_geom_lambda_dih_4int_fix_rep1/epoch=85-val_loss=736.1273-test_loss=21.5217.ckpt'
model = load_model(netfile, device='cpu', derivative=True)
external = external_forces(model, mol, replicas = args.replicas, device = args.device)

In [ ]:
# Define forces and parameters
forces = setup_forces(mol, args.forcefield, args.forceterms, external, device=args.device, 
                        cutoff=args.cutoff, rfa=args.rfa, switch_dist=args.switch_dist
                        )   
# System
system = setup_system(mol, forces, replicas=args.replicas, T=args.temperature,
                        device=args.device
                         )

In [ ]:
steps = 2000
output_period = 10
# Forward pass            
native_coords, system = forward(system, forces, steps, output_period, args.timestep, device=args.device,
                                gamma=args.langevin_gamma, T=args.langevin_temperature
                               )

100%|█████████████████████████████████████████| 200/200 [01:18<00:00,  2.56it/s]


In [ ]:
loss, rmsd = loss_fn(native_coords, system.pos)
print('Final RMSD: ', rmsd)

Final RMSD:  4.733124841662051


## Train Chingolin on a NN pretrained on Lambda proteins

In [ ]:
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

mol = Molecule('../data/chignolin_cln025.pdb')
mol.filter('name CA')
mol.read('../data/cln_ca_top_dih.psf')
mol.read('../data/chignolin_ca_initial_coords.xtc')

train_set = [mol]*10
val_set = []

2021-09-01 17:45:54,085 - moleculekit.molecule - INFO - Removed 83 atoms. 10 atoms remaining in the molecule.


In [ ]:
train_set *= 10
len(train_set)

100

In [ ]:
# Hparams    
hparams = {'epochs': args.num_epochs,
           'max_steps': args.max_steps,
           'step_update': args.step_update, 
           'output_period': 1,
           'lr': args.lr
          }

# Define the NN model
netfile = '/workspace2/fast_folders_cgnet/multiprotein/single_prot_fix/lambda/train_geom_lambda_dih_4int_fix_rep1/epoch=85-val_loss=736.1273-test_loss=21.5217.ckpt'
lambda_nn = load_model(netfile, device='cpu', derivative=True)

optim = torch.optim.Adam(lambda_nn.parameters(), lr=hparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.8)

# Train the model
train_model(lambda_nn, optim, scheduler , hparams, train_set, val_set, args)

100%|█████████████████████████████████████████| 250/250 [00:10<00:00, 23.63it/s]


Example 0 chignolin_ca_top, RMSD 1.4732377771193104


100%|█████████████████████████████████████████| 250/250 [00:09<00:00, 27.60it/s]


Example 1 chignolin_ca_top, RMSD 1.2812741539595693


100%|█████████████████████████████████████████| 250/250 [00:09<00:00, 25.71it/s]


...

100%|█████████████████████████████████████████| 250/250 [00:10<00:00, 23.81it/s]


Example 6 chignolin_ca_top, RMSD 0.8487520665629258


100%|█████████████████████████████████████████| 250/250 [00:19<00:00, 12.74it/s]


Example 7 chignolin_ca_top, RMSD 0.8880901264635022


 43%|█████████████████▋                       | 108/250 [00:23<01:25,  1.67it/s]

## Analysis

In [ ]:
out_dir = 'cln_64trajs_350_ts1'
ts = 0.001
n_traj = 10

fig, ax = plt.subplots(nrows = n_traj, ncols = 2, figsize=[20, 2*n_traj])

for n, axi in enumerate(ax):
    
    # Plot energy from monitor file
    df = pd.read_csv(f'{out_dir}/monitor_{n}.csv')
    df.plot(x='ns', y=['epot', 'ekin', 'etot'], ax =axi[0])
    axi[0].set_ylabel('energy')
    axi[0].legend()
    axi[0].set_ylim([-50, 30])
    
    # now plot the RMSD to refernece molecule 
    
    # load a reference from data/chignolin_cln025.pdb file and filter only CA atoms
    mol_ref = Molecule('data/chignolin_cln025.pdb')
    mol_ref.filter('name CA')
    
    # load an array with trajectory and replace the coord array in molecule object
    mol = Molecule('data/cln_ca_top_dih.psf')
    arr = np.load(f'{out_dir}/output_{n}.npy')
    mol.coords = arr.astype(np.float32)

    # align trajectory (mol) to the reference (mol_ref)
    mol.align('name CA', refmol=mol_ref)
    
    # compute RMSD with MetricRmsd
    rmsd_proj = MetricRmsd(mol_ref, 'name CA', centerstr='name CA', pbc=False)
    rmsd = rmsd_proj.project(mol)
    
    # plot the result
    axi[1].plot(np.arange(len(rmsd))*ts, rmsd, label='True')

    # repeat the process for mirror image of the molecule.
    mul = np.ones_like(arr)
    mul[:,2,:] = -1
    arr *= mul
    
    # replace the coordinates in mol object
    mol.coords = (arr).astype(np.float32)
    
    # align
    mol.align('name CA', refmol=mol_ref)
    
    # compute RMSD
    rmsd_proj = MetricRmsd(mol_ref, 'name CA', centerstr='name CA', pbc=False)
    rmsd = rmsd_proj.project(mol)
    
    # plot
    axi[1].plot(np.arange(len(rmsd))*ts, rmsd, label = 'Mirror', alpha=0.5)
    axi[1].set_ylim([0,8])
    axi[1].set_xlim([0,10])
    axi[1].set_xlabel('ns')
    axi[1].set_ylabel('RMSD')
    axi[1].legend()
    
plt.show()


## Load NN

In [ ]:
from torchmdnet.calculators import External
import importlib

import argparse
import importlib
from moleculekit.molecule import Molecule
from moleculekit.projections.metricrmsd import MetricRmsd
import os
import torch
from torchmd_cg.utils.psfwriter import pdb2psf_CA
from torchmd.utils import save_argparse, LogWriter,LoadFromFile
from torchmd.forcefields.ff_yaml import YamlForcefield
from torchmd.forcefields.forcefield import ForceField
from torchmd.forces import Forces
from torchmd.integrator import Integrator, maxwell_boltzmann
from torchmd.parameters import Parameters
from torchmd.systems import System
from torchmd.wrapper import Wrapper
from tqdm import tqdm
from utils import rmsd


In [ ]:
import torch
from torchmdnet.models.model import load_model


class External:
    def __init__(self, netfile, embeddings, device="cpu"):
        self.model = load_model(netfile, device=device, derivative=True)
        self.device = device
        self.n_atoms = embeddings.size(1)
        self.embeddings = embeddings.reshape(-1).to(device)
        self.batch = torch.arange(embeddings.size(0), device=device).repeat_interleave(
            embeddings.size(1)
        )
        self.model.eval()

    def calculate(self, pos, box):
        pos = pos.to(self.device).type(torch.float32).reshape(-1, 3)
        energy, forces = self.model(self.embeddings, pos, self.batch)
        return energy.detach(), forces.reshape(-1, self.n_atoms, 3)


In [ ]:
# Molecule
mol = Molecule('data/chignolin_cln025.pdb')
mol.filter('name CA')
mol.read('data/chignolin_ca_top.psf')
mol.read('data/chignolin_ca_initial_coords.xtc')

2021-08-24 17:00:58,127 - moleculekit.molecule - INFO - Removed 83 atoms. 10 atoms remaining in the molecule.


In [69]:
# Loading the model as external force

embeddings = [4, 4, 5, 8, 6, 13, 2, 13, 7, 4]
#model = torch.load('data/epoch=47-val_loss=739.5398-test_loss=21.5975.ckpt')
device = 'cpu'

externalmodule = importlib.import_module('torchmdnet.calculators')
embeddings = torch.tensor(embeddings).repeat(10, 1)
external = External('data/epoch=47-val_loss=739.5398-test_loss=21.5975.ckpt', embeddings, device)

In [70]:
ff = ForceField.create(mol,'/shared/carles/torchMD-DMS/nn/data/ca_priors-dihedrals_general_2xweaker.yaml')
cln_parameters = Parameters(ff, mol, terms=['bonds', 'repulsionCG', 'dihedrals'], device=device)
forces = Forces(cln_parameters, terms=['bonds', 'repulsionCG', 'dihedrals'], external=external, cutoff=None, 
                rfa=False, switch_dist=None
                )

In [71]:
# System
system = System(mol.numAtoms, nreplicas=10,precision=torch.double, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(forces.par.masses, T=350, replicas=10))

In [72]:
integrator = Integrator(system, forces, 1, device, gamma=0.1, T=0)
native_coords = system.pos.clone()
ref_coords = mol.coords.copy()

In [73]:
# hparameters

steps = 200
output_period = 10
n_epochs = 20
iterator = tqdm(range(1,int(steps/output_period)+1))

learning_rate = 0.1
optim = torch.optim.Adam(forces.external.model.parameters(), lr=learning_rate)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

In [74]:
train_losses = []
for epoch in range(1, n_epochs + 1):
            
    # Starting...

    system = System(mol.numAtoms, nreplicas=10,precision=torch.double, device=device)
    system.set_positions(mol.coords)
    system.set_box(mol.box)
    system.set_velocities(maxwell_boltzmann(forces.par.masses, T=350, replicas=10))

    Epot = forces.compute(system.pos, system.box, system.forces)
    
    
    for i in iterator:
        Ekin, Epot, T = integrator.step(niter=output_period)
        #currpos = system.pos.detach().cpu().numpy().copy()
    
    # Update params
    
    loss, passed = rmsd(native_coords[0], system.pos.clone()[0])
    train_losses.append(loss)
    loss_log = torch.log(1.0 + loss)
    
    optim.zero_grad()
    loss_log.backward()
    optim.step()
    
    del system
    print(f'Epoch {epoch}, Training loss {loss.item()}')
        
    
    #for rep, coords in enumerate(native_coords):
    #    print('RMSD GOOD: ', rmsd(native_coords[rep], currpos[rep]))
    

100%|███████████████████████████████████████████| 20/20 [00:16<00:00,  1.24it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [26]:
ref_coords = mol.coords.copy()
ref_coords

array([[[ -6.88      ,  -6.88      ,  -6.88      ,  -6.88      ,
          -6.88      ,  -9.8       ,  -9.8       ,  -9.8       ,
          -9.8       ,  -9.8       ],
        [ -0.71000004,  -0.71000004,  -0.71000004,  -0.71000004,
          -0.71000004,  11.51      ,  11.51      ,  11.51      ,
          11.51      ,  11.51      ],
        [  2.9       ,   2.9       ,   2.9       ,   2.9       ,
           2.9       ,   1.26      ,   1.26      ,   1.26      ,
           1.26      ,   1.26      ]],

       [[ -4.1900005 ,  -4.1900005 ,  -4.1900005 ,  -4.1900005 ,
          -4.1900005 ,  -8.52      ,  -8.52      ,  -8.52      ,
          -8.52      ,  -8.52      ],
        [ -0.3       ,  -0.3       ,  -0.3       ,  -0.3       ,
          -0.3       ,   7.8500004 ,   7.8500004 ,   7.8500004 ,
           7.8500004 ,   7.8500004 ],
        [  0.21000001,   0.21000001,   0.21000001,   0.21000001,
           0.21000001,   1.72      ,   1.72      ,   1.72      ,
           1.72      ,   1.7

In [16]:
for i in iterator:
         #viewFrame(mol, system.pos, system.forces)
    Ekin, Epot, T = integrator.step(niter=output_period)
#wrapper.wrap(system.pos, system.box)
    currpos = system.pos.clone()


100%|███████████████████████████████████████████| 20/20 [00:13<00:00,  1.53it/s]


In [17]:
rmsd(native_coords[0], currpos[0])

(tensor(1.6575, dtype=torch.float64), True)